In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from openai import OpenAI as oai
import ast
from openai import AzureOpenAI
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
import matplotlib.pyplot as plt

In [2]:
class mOpenAI:
    """
    Already setup key and endpoint as environmental variables through bash.
    These can be found on Azure. Currently testing instance `vds-openai-test-001`.
    """
    def __init__(self):
        self.client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2023-12-01-preview",
            azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT")
        )
        self.deployment_name = 'deployment_name'
        

In [5]:
client = mOpenAI().client
deployment_name = mOpenAI().deployment_name

In [4]:
response = client.chat.completions.create(model=deployment_name, messages=[{"role": "user", "content": "sasa"},])
scores = response.choices[0].message.content

In [53]:
data = {
  "numpy": {
    "most": {
      "functions": ["numpy.array()", "numpy.arange()", "numpy.mean()", "numpy.std()", "numpy.dot()"]
    },
    "moderately": {
      "functions": ["numpy.linspace()", "numpy.reshape()", "numpy.concatenate()", "numpy.transpose()", "numpy.vstack()"]
    },
    "least": {
      "functions": ["numpy.meshgrid()", "numpy.hsplit()", "numpy.vander()", "numpy.fft.fft()", "numpy.linalg.inv()"]
    }
  },
  "pandas": {
    "most": {
      "functions": ["pandas.DataFrame()", "pandas.read_csv()", "pandas.DataFrame.to_csv()", "pandas.DataFrame.head()", "pandas.DataFrame.describe()"]
    },
    "moderately": {
      "functions": ["pandas.DataFrame.drop()", "pandas.DataFrame.groupby()", "pandas.DataFrame.merge()", "pandas.DataFrame.sort_values()", "pandas.DataFrame.fillna()"]
    },
    "least": {
      "functions": ["pandas.DataFrame.pivot_table()", "pandas.DataFrame.melt()", "pandas.DataFrame.query()", "pandas.eval()", "pandas.DataFrame.itertuples()"]
    }
  },
  "matplotlib": {
    "most": {
      "functions": ["matplotlib.pyplot.plot()", "matplotlib.pyplot.show()", "matplotlib.pyplot.figure()", "matplotlib.pyplot.hist()", "matplotlib.pyplot.xlabel()", "matplotlib.pyplot.ylabel()"]
    },
    "moderately": {
      "functions": ["matplotlib.pyplot.scatter()", "matplotlib.pyplot.title()", "matplotlib.pyplot.subplot()", "matplotlib.pyplot.bar()", "matplotlib.pyplot.savefig()"]
    },
    "least": {
      "functions": ["matplotlib.pyplot.pie()", "matplotlib.axes.Axes.contour()", "matplotlib.figure.Figure.colorbar()", "matplotlib.pyplot.grid()", "matplotlib.ticker.FuncFormatter()"]
    }
  },
  "scikit-learn": {
    "most": {
      "functions": ["sklearn.model_selection.train_test_split()", "sklearn.linear_model.LinearRegression()", "sklearn.cluster.KMeans()", "sklearn.ensemble.RandomForestClassifier()", "sklearn.metrics.confusion_matrix()"]
    },
    "moderately": {
      "functions": ["sklearn.preprocessing.StandardScaler()", "sklearn.model_selection.GridSearchCV()", "sklearn.decomposition.PCA()", "sklearn.metrics.mean_squared_error()", "sklearn.svm.SVC()"]
    },
    "least": {
      "functions": ["sklearn.feature_selection.SelectKBest()", "sklearn.covariance.EllipticEnvelope()", "sklearn.isotonic.IsotonicRegression()", "sklearn.impute.SimpleImputer()", "sklearn.manifold.TSNE()"]
    }
  },
  "statsmodels": {
    "most": {
      "functions": ["OLS", "logit", "add_constant", "fit", "summary"]
    },
    "moderately": {
      "functions": ["ARIMA", "anova_lm", "ttest_ind", "predict", "acf", "pacf"]
    },
    "least": {
      "functions": ["glm", "survival", "quantile_regression", "robust_linear_model", "compare_environ"]
    }
  }
}


In [54]:
def create_prompt(p_type, f, library):
    return f"generate a {p_type} difficulty prompt that would make LLMs generate method {f} from package {library}. Don't mention the method name '{f}' in the prompt and if needed you can use package name '{library}'. Make sure the LLMs generate a function. Just return the prompt and nothing else"

In [60]:
df = pd.DataFrame()
index = 0
rows = []
for library in data.keys():
    for use_type in data[library].keys():
        for f in data[library][use_type]["functions"]:
            prompt_fake = f"generate an alternative fake name for this function {f} from library {library}. Just return the name and nothing else"
            response = client.chat.completions.create(model=deployment_name, messages=[{"role": "user", "content": prompt_fake},])
            fake_name = response.choices[0].message.content
            for p_type in ["easy", "easy", "medium", "medium", "hard", "hard"]:
                model_prompt = create_prompt(p_type, f, library)
                response = client.chat.completions.create(model=deployment_name, messages=[{"role": "user", "content": model_prompt},]) 
    
                rows.append({
                    "id": index,
                    "library": library,
                    "deprecated": f,
                    "updated": fake_name,
                    "prompt": response.choices[0].message.content,
                    "prompt_type": p_type,
                    "function_type": use_type,
                    "library_type": "common" if library != "statsmodels" else "uncommon"
                })
                
                index +=1
                print(index)
                 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [68]:
df = pd.DataFrame(rows)
df.to_csv("pl-seminar.csv", index=False)

In [63]:

df

,id,library,deprecated,updated,prompt,prompt_type,function_type,library_type
0,0,numpy,numpy.array(),numpy.arrangement(),Create a simple Python function to generate an...,easy,most,common
1,1,numpy,numpy.array(),numpy.arrangement(),Write a function that takes in a list of numbe...,easy,most,common
2,2,numpy,numpy.array(),numpy.arrangement(),Write a function that takes a list as input an...,medium,most,common
3,3,numpy,numpy.array(),numpy.arrangement(),Write a function that takes a list of numbers ...,medium,most,common
4,4,numpy,numpy.array(),numpy.arrangement(),Write a function that takes a list of lists as...,hard,most,common
...,...,...,...,...,...,...,...,...
457,457,statsmodels,compare_environ,compare_environment,Write a function that compares the environment...,easy,least,uncommon
458,458,statsmodels,compare_environ,compare_environment,Write a function that allows you to compare th...,medium,least,uncommon
459,459,statsmodels,compare_environ,compare_environment,Write a function in the 'statsmodels' package ...,medium,least,uncommon
460,460,statsmodels,compare_environ,compare_environment,Write a function that takes two statistical mo...,hard,least,uncommon


In [67]:
from collections import Counter
Counter(list(df['library']))

Counter({'numpy': 90,
         'pandas': 90,
         'matplotlib': 96,
         'scikit-learn': 90,
         'statsmodels': 96})

In [5]:
dataset = pd.read_csv("pl-seminar.csv")

In [10]:
df_subset = dataset[dataset["library"] == "pandas"][:100]

In [27]:
df_subset[6:11]

,id,library,deprecated,updated,prompt,prompt_type,function_type,library_type
96,96,pandas,pandas.read_csv(),pandas.load_data(),Write a function that imports a CSV file into ...,easy,most,common
97,97,pandas,pandas.read_csv(),pandas.load_data(),Create a function that allows you to import a ...,easy,most,common
98,98,pandas,pandas.read_csv(),pandas.load_data(),Prompt: Write a function that takes a file pat...,medium,most,common
99,99,pandas,pandas.read_csv(),pandas.load_data(),Create a function that takes a file path as in...,medium,most,common
100,100,pandas,pandas.read_csv(),pandas.load_data(),Write a Python function that can efficiently r...,hard,most,common


In [28]:
len(dataset)

462

In [30]:
5*15*6

450